In [10]:
import requests
import os

def api_request(raw_image, heatmap_image):
    url = "http://127.0.0.1:8000/upload/"

    files = {}

    # Handle raw image
    raw_image_extension = os.path.splitext(raw_image.name)[1].lower()
    files['image_raw'] = ('image_raw' + raw_image_extension, raw_image.read(), f'image/{raw_image_extension[1:]}')

    # Handle heatmap image
    heatmap_image_extension = os.path.splitext(heatmap_image.name)[1].lower()
    files['image_heatmap'] = ('image_heatmap' + heatmap_image_extension, heatmap_image.read(), f'image/{heatmap_image_extension[1:]}')

    try:
        response = requests.post(url, files=files)
        response.raise_for_status()

        json_data = response.json()
        formatted_text = format_json(json_data)

        return formatted_text

    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

def format_json(data):
    formatted_text = ""
    for key, value in data.items():
        formatted_text += f"**{key}**: {value}\n\n"
    return formatted_text.strip()


In [ ]:
import gradio as gr
import requests
from io import BytesIO
from PIL import Image
from IPython.display import display, Markdown

# Function to format JSON response as Markdown
def format_json(json_data):
    formatted = ""
    for key, value in json_data.items():
        formatted += f"**{key}:**\n{value}\n\n"
    return formatted.strip()

# Replace with the actual URL of your FastAPI endpoint
url = "http://127.0.0.1:8000/upload/"

def predict(raw_image, heatmap_image):
    try:
        # Prepare files
        files = {
            'image_raw': ('image.png', raw_image, 'image/png'),
            'image_heatmap': ('heatmap.png', heatmap_image, 'image/png')
        }

        # Send POST request with files
        response = requests.post(url, files=files)
        response.raise_for_status()

        # Format and display JSON response
        json_data = response.json()
        formatted_response = format_json(json_data)
        display(Markdown(formatted_response))

    except Exception as e:
        return f"Error: {str(e)}"

# Define the Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(type="pil", label="Upload Raw Image"),
        gr.Image(type="pil", label="Upload Heatmap Image")
    ],
    outputs=gr.Textbox(label="API Response (formatted)")
)

# Display the Gradio interface
iface.launch()
